<a href="https://colab.research.google.com/github/NeuralNetworkNecromancer/Alrik-Abenteurer/blob/main/Alrik_Abenteurer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####INITIATE####
!pip install qdrant-client openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests PyPDF2 pytz

# utils
import os
import json
import openai
from datetime import datetime
from IPython.display import display, clear_output, Markdown, HTML
from google.colab import drive, files, auth, userdata, output
import ipywidgets as widgets
import pytz

#data loading packages
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import PyPDF2
import re

from googleapiclient.http import MediaIoBaseDownload
import io

drive.mount('/content/drive')

# Specify folder and file details
folder_dir = f'/content/drive/MyDrive/DSA/Alrik Abenteurer'
app_dir = f'{folder_dir}/App'
targeted_folder = f'{folder_dir}/Rules'
company_name = 'alrik'

# Setup Openai
OPENAI_API_KEY = userdata.get('openai')
openai.api_key = OPENAI_API_KEY

# OAuth2.0 Authentication
SCOPES = ['https://www.googleapis.com/auth/drive']

# If modifying these SCOPES, delete the file token.json.
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        auth.authenticate_user()
        creds, _ = google.auth.default()

# Build the Drive and Docs services
drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)
# Build the Google Sheets API client.
sheets_service = build('sheets', 'v4', credentials=creds)

clear_output()

In [2]:
####SEARCH FILES####
Frage = "was sind gute nahkampf techniken um mit raufen oder ringen gegen bewaffnete an zu treten?" #@param {type: "string" }
Anzahl_zu_durchsuchender_Dokumente = "10" #@param {type: "string" }


from qdrant_client import QdrantClient
from qdrant_client.http import models

# Setup Qdrant
QDRANT_API_KEY = userdata.get('qdrant')

qdrant_client = QdrantClient(
    url=userdata.get('TBU_qdrant_url'),
    api_key=QDRANT_API_KEY,
)


user_input = Frage

oauth2_service = build('oauth2', 'v2', credentials=creds)

# Get user info
user_info = oauth2_service.userinfo().get().execute()
user_email = user_info['email']

def get_base_rules(document_id, creds):
    """
    Accesses a Google Docs document with a specific ID and retrieves its content.

    Parameters:
    - document_id (str): The ID of the Google Docs document.
    - creds (Credentials): The credentials used to access Google Drive API.

    Returns:
    - str: The content of the Google Docs document.
    """

    # Build the Docs services
    docs_service = build('docs', 'v1', credentials=creds)

    # Retrieve the document using its ID
    doc = docs_service.documents().get(documentId=document_id).execute()

    # Extract content from the document
    content = []
    for element in doc.get('body', {}).get('content', []):
        content.extend([part.get('textRun', {}).get('content', '')
                        for part in element.get('paragraph', {}).get('elements', [])])

    # Combine extracted content into a single string
    content_text = "".join(content)

    return content_text

# Example usage of the get_document_content function
document_id = '1gA3Xicio9d6vZklPQnFE97M_Ajil4vxqJClEQDR4j-8'
base_rules = get_base_rules(document_id, creds)

# Generate a timestamp
def generate_timestamp(timezone="CET"):
    """
    Generate a timestamp in the specified timezone.

    Parameters:
    - timezone: str, the timezone in which the timestamp is generated.

    Returns:
    - str, the timestamp.
    """
    tz = pytz.timezone(timezone)
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

collection_name = f"{company_name}_embeddings"

def query_text(user_input, model="text-embedding-ada-002", top_k=Anzahl_zu_durchsuchender_Dokumente):
    """
    Query the vector database based on user input text.

    Parameters:
    - user_input: str, text input from the user to query against the vector database.
    - model: str, the name of the OpenAI model used for embedding creation.
    - top_k: int, the number of closest matches to retrieve.

    Returns:
    - list of dict, containing the top_k most similar documents to the user input.
    """

    # Generate embedding for user input
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    embedding = client.embeddings.create(
        input=[user_input], model=model)
    user_embedding = embedding.data[0].embedding

    # Perform query to find nearest neighbors in the vector database
    results = qdrant_client.search(
        collection_name=collection_name,
        limit=top_k,  # Retrieve top_k most similar points
        query_vector=user_embedding,
        with_payload=True  # Get the payload data along with the results
    )

    # Extract and return useful information from the results
    similar_docs = [
        {
            "id": result.id,
            "payload": result.payload,
            "score": result.score,
        }
        for result in results
    ]

    return similar_docs


ranked_results = query_text(user_input)

# Print or further process the results as needed
print(json.dumps(ranked_results, indent=4))  # Pretty print the results with indentation

# Extract text content from the payloads into a list
text_contents = []
titles = []
scores = []
for doc in ranked_results:
    payload = doc.get('payload', {})
    text_content = payload.get('content_text', '')  # Replace with actual key if different
    title = payload.get('file_name', '')  # Replace with actual key if different
    score = doc.get('score', 0)

    text_contents.append(text_content)
    titles.append(title)
    scores.append(score)

# Now text_contents is a list of the text content from the ranked_results
print("Extracted text contents:")
print(text_contents)

# Generate an answer based on a file
def generate_answer(user_input, text_contents):
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
            f"role": "system", "content": # setup und high level context
            f"Du bist ein Regelexperte und Spielleiter (eng.: Dungeon Master) einer Pen-&-Paper-Rollenspiel Kampagne im Das Schwarze Auge (DSA) Regelsystem."
            f"Dies sind die Basis Regeln: {base_rules}"
            f"Bitte beantworte die Fragen mit entsprechenden detaillierten Erklärungen wie die Regeln funktionieren und an zu wenden sind."
            f"Bleibe jedoch faktisch genau bei den Regeln und erfinde nichts dazu."
            f"Sofern nicht anders gefordert, antworte kurz und bündig."
            f"Bitte unterlasse jede einschätzung zu deiner eignung der Fragestellung und Antworte ausschließlich bezogen auf die Fragestellung."
            f"Beantworte IMMER die Frage die gestellt wird."
            f"Falls nicht anders vor gegeben antworte mit maximal 3 Sätzen."
            },
            {
              "role": "user", "content": # prompt
             f"Beantworte die Frage: '{user_input}', basierent auf deinem Wissen und diesen Regeln: \n{text_contents} zusätzlich zu den Basis Regeln"
             }
        ],
        max_tokens=1000,
        n=1,
        stop=None,
        temperature=0.7,
    )

    answer = response.choices[0].message.content.strip()
    return answer

answer = generate_answer(user_input, text_contents)

clear_output()

display(Markdown("### Frage: "))
display(Markdown(f"_{user_input}_\n"))
display(Markdown("### Antwort: "))
display(Markdown(f"_{' '.join(answer.split())}_"))


# Function to handle button click
def open_button_clicked(button):
    # Extract the document ID and file name stored in the button's custom attributes
    DOCUMENT_ID = button.doc_id
    FILE_NAME = button.file_path

    # Export Google Docs document as text
    request = drive_service.files().export_media(fileId=DOCUMENT_ID, mimeType='text/plain')
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()

    # Write the exported content to a file
    with open(FILE_NAME, 'wb') as f:
        f.write(fh.getvalue())

    # Now you can read the file
    with open(FILE_NAME, 'r') as file:
        content = file.read()

    print(f"\n\n------------------------------------------------------------------\n\n")
    print(content)

display(Markdown("### Regel Referenzen zum Nachschlagen: "))

# Accumulate all HBox widgets in a list
hboxes = []

for doc, title in zip(ranked_results, titles):
    payload = doc.get('payload', {})
    document_id = payload.get("file_id", "")  # Adjust the key if necessary
    file_name = payload.get("file_name", "")
    file_url = payload.get("file_url", "")
    folder_path = payload.get("folder_path", "")

    # Create a button for each document
    button = widgets.Button(description='öffnen')
    button.doc_id = document_id  # Store document ID in the button object
    button.file_path = f'{targeted_folder}{folder_path}/{file_name}.doc'  # Store file name in the button object
    button.on_click(open_button_clicked)  # Set the event handler

    # Create a hyperlink using widgets.HTML with increased font size
    html_value = f"<a href='{file_url}' target='_blank' style='font-size:14px;'>{file_name}</a>"
    link = widgets.HTML(value=html_value)

    # Create a separator label
    separator = widgets.HTML(value=" | ")

    # Create an HBox for each hyperlink, button pair, and separator
    hbox = widgets.HBox([link, button, separator])
    hboxes.append(hbox)

# Display all HBoxes in a single row
all_hboxes = widgets.HBox(hboxes)
display(all_hboxes)

# Load the existing QA pairs from the JSON file if it exists
try:
    with open(f'{app_dir}/{company_name}_qa_history.json', 'r') as file:
        qa_history = json.load(file)
except FileNotFoundError:
    # If file doesn't exist, initialize an empty list
    qa_history = []

# Append the new QA pair and save to file
qa_pair = {"user": user_email,
           "timestamp": generate_timestamp(),
           "question": Frage,
           "answer": answer,
           "returned_amount": Anzahl_zu_durchsuchender_Dokumente,
           "references": [{"Titel": t, "Score": s} for t, s in zip(titles, scores)]
           }
qa_history.append(qa_pair)

# Save the updated QA pairs back to the JSON file
with open(f'{app_dir}/{company_name}_qa_history.json', 'w') as file:
    json.dump(qa_history, file, ensure_ascii=False, indent=4)

# Confirming the action
display(Markdown("<sub>Logs gespeichert.</sub>"))


### Frage: 

_was sind gute nahkampf techniken um mit raufen oder ringen gegen bewaffnete an zu treten?_


### Antwort: 

_Gute Nahkampftechniken für einen waffenlosen Kämpfer gegen Bewaffnete sind: 1. **Mercenario-Stil**: Dieser Stil erlaubt es dem Kämpfer, gegen Bewaffnete in der Distanzklasse Nah (DK:N) nur einen Malus von drei Punkten auf die Attacke (AT) zu erleiden und keinen Schaden zu nehmen, falls seine AT zur Distanzklassen-Verkürzung pariert wird. 2. **Eisenarm**: Mit der Sonderfertigkeit Eisenarm kann ein Held bewaffnete Attacken waffenlos parieren, ohne dabei Schaden zu erleiden. Zudem ermöglicht diese Fertigkeit das Anwenden von waffenlosen Manövern wie Binden und Entwaffnen gegen Bewaffnete, allerdings mit einer Erschwernis von 4 Punkten. 3. **Schmutzige Tricks**: Mit dieser Fertigkeit kann der Kämpfer die Umgebung nutzen, um Ausdauerschaden oder Initiative-Verlust anzurichten, ohne dass die Rüstung des Gegners den Schaden mindert. Dies kann zum Beispiel durch das Werfen von Gegenständen oder das Anwenden von Umgebungsmanövern geschehen._

### Regel Referenzen zum Nachschlagen: 

<sub>Logs gespeichert.</sub>